In [2]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd
import plotly.io as pio

# Load datasets
matches = pd.read_csv("matches.csv")
deliveries = pd.read_csv("deliveries.csv")

# Matches per Season
matches_per_season = matches['season'].value_counts().sort_index()
season_bar = go.Bar(x=matches_per_season.index, y=matches_per_season.values, name='Matches per Season')

# Winner Distribution (Home vs Away)
home_wins = len(matches[matches['team1'] == matches['winner']])
away_wins = len(matches[matches['team1'] != matches['winner']])
winner_pie = go.Pie(labels=['Home Wins', 'Away Wins'], values=[home_wins, away_wins], name='Winner Distribution')

# Top Batsmen (replace with your desired metric)
top_batsman = (
    deliveries.groupby("batsman")["batsman_runs"]
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
)
batsman_scatter = go.Bar(x=top_batsman['batsman'], y=top_batsman['batsman_runs'], name='Top Batsmen')

# Create a layout for your dashboard
layout = go.Layout(title='IPL Matches Analysis')

# Arrange your plots in a grid layout (adjust rows and columns as needed)
fig = make_subplots(rows=1, cols=2, subplot_titles=("Matches per Season", "Top Batsmen"))

# Add traces to subplots
fig.add_trace(season_bar, row=1, col=1)
fig.add_trace(batsman_scatter, row=1, col=2)

# Display the dashboard
html_file = "ipl_dashboard.html"
pio.write_html(fig, html_file)


In [6]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go

# Load datasets
matches = pd.read_csv("matches.csv")
deliveries = pd.read_csv("deliveries.csv")

# Define app layout
app = dash.Dash(__name__)

# Matches per Season
matches_per_season = matches['season'].value_counts().sort_index()
matches_per_season_fig = go.Figure(
    data=[go.Bar(x=matches_per_season.index, y=matches_per_season.values, marker_color='rgb(255, 102, 102)')],
    layout={'title': 'Matches per Season', 'xaxis_title': 'Season', 'yaxis_title': 'Number of Matches'}
)

# Winner Distribution (Home vs Away)
home_wins = len(matches[matches['team1'] == matches['winner']])
away_wins = len(matches[matches['team1'] != matches['winner']])
winner_distribution_fig = go.Figure(
    data=[go.Pie(labels=['Home Wins', 'Away Wins'], values=[home_wins, away_wins], marker_colors=['rgb(255, 204, 102)', 'rgb(102, 204, 255)'])],
    layout={'title': 'Winner Distribution'}
)

# Top Batsmen by Runs Scored
top_batsmen = (
    deliveries.groupby("batsman")["batsman_runs"]
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
)
top_batsmen_fig = go.Figure(
    data=[go.Bar(x=top_batsmen['batsman'], y=top_batsmen['batsman_runs'], marker_color='rgb(102, 255, 102)')],
    layout={'title': 'Top Batsmen by Runs Scored', 'xaxis_title': 'Batsmen', 'yaxis_title': 'Total Runs Scored'}
)

# Venue Analysis: Matches Played at Each Venue
venue_counts = matches['venue'].value_counts()
venue_fig = go.Figure(
    data=[go.Bar(x=venue_counts.index, y=venue_counts.values, marker_color='rgb(255, 153, 255)')],
    layout={'title': 'Venue Analysis: Matches Played at Each Venue', 'xaxis_title': 'Venue', 'yaxis_title': 'Number of Matches'}
)

# Dismissals Analysis: Types of Dismissals
dismissal_counts = deliveries['dismissal_kind'].value_counts()
dismissals_fig = go.Figure(
    data=[go.Bar(x=dismissal_counts.index, y=dismissal_counts.values, marker_color='rgb(255, 153, 0)')],
    layout={'title': 'Dismissals Analysis: Types of Dismissals', 'xaxis_title': 'Dismissal Type', 'yaxis_title': 'Count'}
)

# Toss Analysis: Impact of Winning Toss on Match Result
toss_impact_fig = go.Figure()
toss_impact_fig.add_trace(go.Box(y=matches[matches['toss_winner'] == matches['winner']]['win_by_runs'], name='Toss Winners', marker_color='rgb(51, 153, 255)'))
toss_impact_fig.add_trace(go.Box(y=matches[matches['toss_winner'] != matches['winner']]['win_by_runs'], name='Toss Losers', marker_color='rgb(255, 51, 51)'))
toss_impact_fig.update_layout(title='Toss Analysis: Impact of Winning Toss on Match Result', yaxis_title='Runs', boxmode='group')

# Team Performance: Win Percentage of Each Team
team_performance = matches.groupby('winner').size() / matches.groupby('team1').size()
team_performance_fig = go.Figure(
    data=[go.Bar(x=team_performance.index, y=team_performance.values * 100, marker_color='rgb(102, 0, 102)')],
    layout={'title': 'Team Performance: Win Percentage of Each Team', 'xaxis_title': 'Team', 'yaxis_title': 'Win Percentage (%)'}
)

# Define app layout
app.layout = html.Div(style={'backgroundColor': '#f0f0f0'}, children=[
    html.H1('IPL Matches Analysis', style={'textAlign': 'center', 'color': '#333'}),
    
    html.Div([
        dcc.Graph(id='matches-per-season', figure=matches_per_season_fig),
        dcc.Graph(id='winner-distribution', figure=winner_distribution_fig)
    ], style={'display': 'flex', 'justifyContent': 'center', 'marginBottom': '30px'}),

    html.Div([
        dcc.Graph(id='top-batsmen', figure=top_batsmen_fig)
    ], style={'textAlign': 'center', 'marginBottom': '30px'}),
    
    html.Div([
        dcc.Graph(id='venue-analysis', figure=venue_fig)
    ], style={'textAlign': 'center', 'marginBottom': '30px'}),

    html.Div([
        dcc.Graph(id='dismissals-analysis', figure=dismissals_fig)
    ], style={'textAlign': 'center', 'marginBottom': '30px'}),

    html.Div([
        dcc.Graph(id='toss-analysis', figure=toss_impact_fig)
    ], style={'textAlign': 'center', 'marginBottom': '30px'}),

    html.Div([
        dcc.Graph(id='team-performance', figure=team_performance_fig)
    ], style={'textAlign': 'center'})
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [9]:
from selenium import webdriver
import time

# Launch a headless Chrome browser
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1200x600')  # Set the window size
driver = webdriver.Chrome(options=options)

# Open the Dash app in the browser
driver.get('http://127.0.0.1:8050/')  # Replace with the URL of your Dash app

# Wait for the app to load
time.sleep(5)  # Adjust the sleep time as needed

# Take a screenshot of the entire page
driver.save_screenshot('dashboard_screenshot.png')

# Close the browser
driver.quit()
